# Configuration

In [2]:
fractionOfData = 0.01

# Import

In [1]:
import pandas as pd
import time
import random
import math
import matplotlib.pyplot as plt

import tensorflow as tf

# Helper functions

In [3]:
%run utils.py

# Load data

In [5]:
metaData_apartment = pd.read_json('../data/apartments_meta.json')
metaData_building = pd.read_json('../data/buildings_meta.json')

In [6]:
metaData_apartment.at[0, 'name'] = 'apartment_id'
metaData_building.at[0, 'name'] = 'building_id'

In [7]:
metaData = pd.concat([metaData_apartment, metaData_building])
metaData

,name,type,desc,cats
0,apartment_id,integer_id,Unique ID for apartment,NaN
1,seller,categorical,The type of apartment seller,"[Owner, Company, Agents, Developer]"
2,price,real,The listed price of the apartment (TARGET),NaN
3,area_total,real,Total area of the apartment,NaN
4,area_kitchen,real,Total kitchen area in the apartment,NaN
5,area_living,real,Total living space area in the apartment,NaN
6,floor,integer,Primary building floor of the apartment,NaN
7,rooms,integer,Number of rooms in the apartment,NaN
8,layout,categorical,Overal apartment layout,"[Adjacent, Isolated, Adjacent_isolated]"
9,ceiling,real,Ceiling height in the apartment,NaN


In [9]:
metaDataDict = {}
for index, row in metaData.iterrows():
    metaDataDict[row['name']] = {'type':list(row)[1], 'categories':list(row)[3]}

In [10]:
metaDataDict['seller']

{'type': 'categorical',
 'categories': ['Owner', 'Company', 'Agents', 'Developer']}

In [11]:
metaDataDict['apartment_id']['categories']

nan

In [13]:
train_apartment = pd.read_csv('../data/apartments_train.csv')

In [14]:
train_apartment.head()

,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,bathrooms_shared,bathrooms_private,windows_court,windows_street,balconies,loggias,condition,phones,building_id
0,0,3.0,7139520.0,59.2,12.5,31.0,2.0,2.0,NaN,2.65,0.0,2.0,0.0,1.0,NaN,NaN,NaN,1.0,4076
1,1,NaN,10500000.0,88.0,14.2,48.0,18.0,3.0,1.0,NaN,2.0,0.0,1.0,1.0,1.0,0.0,3.0,1.0,1893
2,2,3.0,9019650.0,78.5,22.5,40.8,12.0,3.0,NaN,2.65,0.0,2.0,1.0,1.0,NaN,NaN,NaN,1.0,5176
3,3,NaN,10500000.0,88.0,14.0,48.0,18.0,3.0,NaN,NaN,0.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,1893
4,4,NaN,13900000.0,78.0,17.0,35.0,7.0,2.0,1.0,2.90,1.0,0.0,1.0,1.0,0.0,1.0,3.0,1.0,6604


In [15]:
train_building = pd.read_csv('../data/buildings_train.csv')

In [16]:
train_building.head()

,id,new,latitude,longitude,district,street,address,constructed,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating
0,264,0.0,55.518117,37.355139,11.0,улица 1-я Нововатутинская,3,2019.0,2.0,12.0,1.0,1.0,1.0,0.0,1.0,3.0
1,6079,0.0,55.641920,37.352156,7.0,мкр. 10-й,к1014,1981.0,3.0,22.0,0.0,1.0,1.0,NaN,1.0,0.0
2,5829,0.0,55.770011,37.386220,7.0,Рублевское шоссе,68,2002.0,3.0,24.0,0.0,1.0,1.0,NaN,NaN,0.0
3,5758,0.0,55.739934,37.426070,7.0,Рублевское шоссе,93К2,2011.0,3.0,17.0,0.0,1.0,1.0,NaN,1.0,0.0
4,6761,0.0,55.649921,37.467994,7.0,улица Академика Анохина,58К2,2005.0,3.0,22.0,0.0,1.0,1.0,NaN,0.0,0.0


In [17]:
train = pd.merge(train_apartment, train_building, left_on='building_id', right_on='id')
#train.rename(columns={'id_x' : 'apartment_id', 'id_y' : 'building_id'}, inplace=True)
train.rename(columns={'id_x' : 'apartment_id'}, inplace=True)
train.drop('id_y', axis=1, inplace=True)
# Remove duplicate column after merge.

In [18]:
# Reduce amount of data for performance reasons during implementation.
train = train.head(int(train.shape[0] * fractionOfData))

# Data Processing